In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Input,InputLayer,Reshape,GRU,LSTM,Concatenate
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import models,Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D,GlobalAveragePooling2D


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [2]:
class CustomCallback(keras.callbacks.Callback):
    
    def __init__(self, threshold=0.8):
        super(CustomCallback, self).__init__()
        self.threshold = threshold
        
    def on_epoch_end(self, epoch, logs=None):
        if logs["accuracy"]>self.threshold:
            self.model.save_weights('G:/Project Server Pred/model_weights/weights-threshold{}.h5'.format(self.threshold), overwrite=False)
            self.threshold =0.9
            print("80 percent saved")
        if logs["accuracy"]>self.threshold:
            self.model.save_weights('G:/Project Server Pred/model_weights/weights-threshold{}.h5'.format(self.threshold), overwrite=False)
            self.threshold = 1
            print("90 percent saved")
            
def callbacks(name, tensorboard = False,customCallback=False, patience=35,location='G:/Project Server Pred/model_weights/'):
    callbacks = [
        ModelCheckpoint(location+'/weights-{}.h5'.format(str(name)), monitor='val_loss', save_best_only=True, save_weights_only=True),
        EarlyStopping(patience=patience, monitor='val_loss', min_delta=0, mode='min'),
        
    ]
    if tensorboard:
        callbacks.append(TensorBoard(log_dir='./logs/{}'.format(name), histogram_freq=0, write_graph=True, write_images=True))
    if customCallback:
        callbacks.append(CustomCallback())
    return callbacks

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def normalize(x):
    x_reshaped = x.reshape(-1,x.shape[-1])
    mean,std=x_reshaped.mean(axis=0),x_reshaped.std(axis=0)
    x_reshaped = (x_reshaped - mean)/std
    print(mean,std)
    return x_reshaped.reshape(x.shape)    #,mean,std

def reshape_Data(x,channel_first=True):   
    
    if channel_first:
        return x.reshape((x.shape[0],1,x.shape[1],x.shape[2]))
    else:
        return x.reshape((x.shape[0],x.shape[1],x.shape[2],1))
    
def reshapeBack_Data(x,channel_first=True):
    
    if channel_first:
        return x.reshape((x.shape[0],x.shape[2],x.shape[3]))
    else:
        return x.reshape((x.shape[0],x.shape[1],x.shape[2]))
    
def my_PCA(data,gravity_axis=1):
    data_n = data.copy()
    for i,d in enumerate(data_n):
        pca = PCA()
        d = np.delete(d[:,:3],gravity_axis,axis=1)
        pca.fit(d)
        data_t = pca.transform(d)
        
        if gravity_axis==1:
            data_n[i][:,0] = data_t[:,0]
            data_n[i][:,2] = data_t[:,1]
        else:
            print("need to add code")
    return data_n




In [4]:
def LSTM_CNN_SANDWICH(input_shape,output_shape):
    K.set_image_data_format('channels_last')
    n_outputs = output_shape
    input_shape = input_shape
    
    input           = Input(shape=input_shape)
    lstm_1          = Bidirectional(LSTM(32,return_sequences=True))(input)
    activation_1    = Activation("relu")(lstm_1)
    reshape_layer_1 = Reshape((activation_1.shape[1],activation_1.shape[2],1))(activation_1)
    cnn_1           = Conv2D(64, (5,1), strides=(2,1), activation='relu')(reshape_layer_1)
    max_pool_1      = MaxPooling2D((2,1), strides=(2,1))(cnn_1)
    cnn_1_2         = Conv2D(32, (5,1), strides=(2,1), activation='relu')(max_pool_1)
    max_pool_1_2    = MaxPooling2D((2,1), strides=(2,1))(cnn_1_2)
    batch_norm_1    = BatchNormalization()(max_pool_1_2)

    reshape_layer_2 = Reshape((-1,cnn_1.shape[3]))(batch_norm_1)
    lstm_2          = Bidirectional(LSTM(32,return_sequences=True))(reshape_layer_2)
    activation_2    = Activation("relu")(lstm_2)
    reshape_layer_3 = Reshape((activation_2.shape[1],activation_2.shape[2],1))(activation_2)
    cnn_2           = Conv2D(64, (5,1), strides=(2,1), activation='relu')(reshape_layer_3)
    max_pool_2      = MaxPooling2D((2,1), strides=(2,1))(cnn_2)
    cnn_2_2         = Conv2D(32, (5,1), strides=(2,1), activation='relu')(max_pool_2)
    max_pool_2_2    = MaxPooling2D((2,1), strides=(2,1))(cnn_2_2)
    batch_norm_2    = BatchNormalization()(max_pool_2_2)

    global_avg      = GlobalAveragePooling2D()(batch_norm_2)

    dense           = Dense(n_outputs)(global_avg)
    activation_3    = Activation("softmax")(dense)

    model = Model(inputs = input, outputs= activation_3)
    model.summary()

    rmsprop = keras.optimizers.RMSprop(learning_rate=0.0001)

    model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy',f1_m])

    return model
    
LSTM_CNN_SANDWICH((128,9),9)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 9)]          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128, 64)           10752     
_________________________________________________________________
activation_2 (Activation)    (None, 128, 64)           0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 128, 64, 1)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 64, 64)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 64, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 64, 32)        1027

In [5]:
def LSTM_CNN(input_shape=None,n_outputs=None):
    
    K.set_image_data_format('channels_last')
    n_outputs = n_outputs
    input_shape = input_shape
    input           = Input(shape=input_shape)

    lstm_1          = Bidirectional(LSTM(32,return_sequences=True))(input)
    activation_1    = Activation("relu")(lstm_1)
    lstm_2          = Bidirectional(LSTM(32,return_sequences=True))(activation_1)
    activation_2    = Activation("relu")(lstm_2)

    reshape_layer_1 = Reshape((lstm_2.shape[1],lstm_2.shape[2],1))(activation_2)
    
    cnn_1           = Conv2D(64, (5,5), strides=(2,2), activation='relu')(reshape_layer_1)
    max_pool_1      = MaxPooling2D((2,2), strides=(2,2))(cnn_1)
    cnn_2           = Conv2D(128, (3,3), strides=(1,1), activation='relu')(max_pool_1)
    global_avg      = GlobalAveragePooling2D()(cnn_2)
    dense           = Dense(n_outputs,activation='softmax',kernel_regularizer=keras.regularizers.l2(0.005))(global_avg)

    model = Model(inputs = input, outputs= dense)
    model.summary()

    rmsprop = keras.optimizers.RMSprop(learning_rate=0.001)

    model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy',f1_m])
    
    
    return model

LSTM_CNN((128,9),9)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 9)]          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128, 64)           10752     
_________________________________________________________________
activation_5 (Activation)    (None, 128, 64)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128, 64)           24832     
_________________________________________________________________
activation_6 (Activation)    (None, 128, 64)           0         
_________________________________________________________________
reshape_4 (Reshape)          (None, 128, 64, 1)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 62, 30, 64)        1664

In [6]:
def ConvLSTM_PARALLEL(input_shape=None,n_outputs=None):
    
    K.set_image_data_format('channels_last')
    input_shape = input_shape
    input           = Input(shape=input_shape)

    lstm_1          = Bidirectional(LSTM(32,return_sequences=True,activation='relu'))(input)
    activation_1    = Activation("relu")(lstm_1)
    lstm_2          = Bidirectional(LSTM(32,return_sequences=True,activation ='relu'))(activation_1)
    activation_2    = Activation("relu")(lstm_2)
    reshape_layer_1 = Reshape((lstm_2.shape[1],lstm_2.shape[2],1))(activation_2)
    
    cnn_1           = Conv2D(64, (5,5), strides=(2,2), activation='relu')(reshape_layer_1)
    max_pool_1      = MaxPooling2D((2,2), strides=(2,2))(cnn_1)
    cnn_2           = Conv2D(128, (3,3), strides=(1,1), activation='relu')(max_pool_1)
    global_avg      = GlobalAveragePooling2D()(cnn_2)
    flatten         = Flatten()(global_avg)

    reshape_2         = Reshape((input.shape[1],input.shape[2],1))(input)
    cnn_1_2           = Conv2D(64, (5,5), strides=(2,2), activation='relu')(reshape_2)
    #max_pool_1_2     = MaxPooling2D((2,2), strides=(2,2))(cnn_1_2)
    cnn_2_2           = Conv2D(128, (3,3), strides=(1,1), activation='relu')(cnn_1_2)
    global_avg_2      = GlobalAveragePooling2D()(cnn_2_2)

    reshape_layer_1_2 = Reshape((global_avg_2.shape[-1],1))(global_avg_2)
    lstm_1_2          = Bidirectional(LSTM(32,return_sequences=True,activation='relu'))(reshape_layer_1_2)
    activation_1_2    = Activation("relu")(lstm_1_2)
    lstm_2_2          = Bidirectional(LSTM(32,return_sequences=True,activation ='relu'))(activation_1_2)
    activation_2_2    = Activation("relu")(lstm_2_2)
    flatten_2         = Flatten()(activation_2_2)

    concatenate       = Concatenate()([flatten,flatten_2])
    dense             = Dense(n_outputs,activation='softmax',kernel_regularizer=keras.regularizers.l2(0.005))(concatenate)



    model = Model(inputs = input, outputs= dense)
    model.summary()

    rmsprop = keras.optimizers.RMSprop(learning_rate=0.0001)

    model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy',f1_m])
    
    
    return model

ConvLSTM_PARALLEL((128,9),9)


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 9)]     0                                            
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 128, 64)      10752       input_4[0][0]                    
__________________________________________________________________________________________________
reshape_6 (Reshape)             (None, 128, 9, 1)    0           input_4[0][0]                    
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 128, 64)      0           bidirectional_6[0][0]            
____________________________________________________________________________________________

In [7]:
def LSTM_CNN_PARALLEL(input_shape=None,n_outputs=None):
    
    K.set_image_data_format('channels_last')
    input_shape = input_shape
    input           = Input(shape=input_shape)

    lstm_1          = Bidirectional(LSTM(32,return_sequences=True,activation='relu'))(input)
    activation_1    = Activation("relu")(lstm_1)
    lstm_2          = Bidirectional(LSTM(32,return_sequences=True,activation ='relu'))(activation_1)
    activation_2    = Activation("relu")(lstm_2)
    flatten         = Flatten()(activation_2)

    reshape_2         = Reshape((input.shape[1],input.shape[2],1))(input)
    cnn_1_2           = Conv2D(64, (5,5), strides=(2,2), activation='relu')(reshape_2)
    #max_pool_1_2     = MaxPooling2D((2,2), strides=(2,2))(cnn_1_2)
    cnn_2_2           = Conv2D(128, (3,3), strides=(1,1), activation='relu')(cnn_1_2)
    global_avg_2      = GlobalAveragePooling2D()(cnn_2_2)

    flatten_2         = Flatten()(global_avg_2)

    concatenate       = Concatenate()([flatten,flatten_2])
    dense             = Dense(n_outputs,activation='softmax',kernel_regularizer=keras.regularizers.l2(0.005))(concatenate)



    model = Model(inputs = input, outputs= dense)
    model.summary()

    rmsprop = keras.optimizers.RMSprop(learning_rate=0.0001)

    model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy',f1_m])
    
    
    return model

LSTM_CNN_PARALLEL((128,9),9)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 9)]     0                                            
__________________________________________________________________________________________________
bidirectional_10 (Bidirectional (None, 128, 64)      10752       input_5[0][0]                    
__________________________________________________________________________________________________
reshape_8 (Reshape)             (None, 128, 9, 1)    0           input_5[0][0]                    
__________________________________________________________________________________________________
activation_11 (Activation)      (None, 128, 64)      0           bidirectional_10[0][0]           
____________________________________________________________________________________________

In [6]:
def GRU_CNN(input_shape,output_shape):
    K.set_image_data_format('channels_last')
    
    n_outputs = output_shape
    input_shape = input_shape
    
    input = Input(shape = input_shape)
    gru_1 = GRU(32, return_sequences = True, activation = 'relu')(input)
    gru_2 = GRU(32, return_sequences = True, activation = 'relu')(gru_1)
    rehsape_layer_1 = Reshape((gru_2.shape[1],gru_2.shape[2],1))(gru_2)
    cnn_1 = Conv2D(64, (5,5), strides=(2,2), activation='relu')(rehsape_layer_1)
    max_pool_1 = MaxPooling2D((2,2), strides=(2,2)) (cnn_1)
    cnn_2 = Conv2D(128, (3,3), strides=(1,1), activation='relu')(max_pool_1)

    global_avg = GlobalAveragePooling2D()(cnn_2)
    batch_norm = BatchNormalization()(global_avg)

    dense = Dense(n_outputs,activation ='softmax') (batch_norm)

    model = Model(inputs = input,outputs = dense)
    model.summary()
    rmsprop=keras.optimizers.RMSprop(learning_rate=0.001)
    model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy',f1_m])

    return model

GRU_CNN((128,9),9)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 9)]          0         
_________________________________________________________________
gru (GRU)                    (None, 128, 32)           4128      
_________________________________________________________________
gru_1 (GRU)                  (None, 128, 32)           6336      
_________________________________________________________________
reshape (Reshape)            (None, 128, 32, 1)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 62, 14, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 7, 64)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 5, 128)        73856 

In [12]:
def CNN_GRU(input_shape,output_shape):
    K.set_image_data_format('channels_last')
    
    n_outputs = output_shape
    input_shape = input_shape
    
    input   = Input(shape=input_shape)
    reshape_layer = Reshape((input.shape[1],input.shape[2],1)) (input)
    cnn_1         = Conv2D(128, (5,1), strides=(1,1), activation='relu') (reshape_layer)
    max_pool_1    = MaxPooling2D((2,1)) (cnn_1)
    cnn_2         = Conv2D(64, (3,1), strides=(1,1), activation='relu') (max_pool_1)
    max_pool_2    = MaxPooling2D((2,1)) (cnn_2)
    cnn_3         = Conv2D(32, (3,2), strides=(1,1), activation='relu') (max_pool_2)
    max_pool_3    = MaxPooling2D((2,1)) (cnn_3)
    batch_norm    = BatchNormalization()(max_pool_3)

    reshape_layer_2 = Reshape((-1,batch_norm.shape[3]))(batch_norm)
    gru_1 = GRU(32, return_sequences = True, activation = 'relu')(reshape_layer_2)
    gru_2 = GRU(32, activation = 'relu')(gru_1)

    #global_avg = GlobalAveragePooling2D()(gru_2)
    batch_norm = BatchNormalization()(gru_2)

    dense = Dense(n_outputs,activation ='softmax') (batch_norm)

    model = Model(inputs = input,outputs = dense)

    model.summary()

    rmsprop=keras.optimizers.RMSprop(learning_rate=0.001)
    model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy',f1_m])
    
    return model

CNN_GRU((128,9),9)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 128, 9)]          0         
_________________________________________________________________
reshape_13 (Reshape)         (None, 128, 9, 1)         0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 124, 9, 128)       768       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 62, 9, 128)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 60, 9, 64)         24640     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 30, 9, 64)         0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 28, 8, 32)         1232

In [11]:
import pickle
import numpy as np

def normalize(x,mean = None,std = None):
    x_reshaped = x.reshape(-1,x.shape[-1])
    if mean is None or std is None:
        mean,std   = x_reshaped.mean(axis=0),x_reshaped.std(axis=0)
    x_reshaped = (x_reshaped - mean)/std
    print(mean,std)
    return x_reshaped.reshape(x.shape),mean,std    #,mean,std

location = "G:/Project Server Pred/Dataset/"



trainX = pickle.load(open(location+"UCI_ALL_trainX_7act", "rb"))
trainY = pickle.load(open(location+"UCI_ALL_trainY_7act", "rb"))

valX = pickle.load(open(location+"UCI_ALL_valX_7act", "rb"))
valY = pickle.load(open(location+"UCI_ALL_valY_7act", "rb"))

    


X_train,m,s = normalize(trainX)
X_val,_,_   = normalize(valX,m,s)
Y_train     = to_categorical(trainY-1)
Y_val       = to_categorical(valY-1)

index = np.arange(len(X_train))
np.random.shuffle(index)
X_train = X_train[index]
Y_train = Y_train[index]

print(X_train.shape,Y_train.shape,X_val.shape,Y_val.shape)


[ 5.73906856e-04 -6.18831527e-05  1.69143052e-04  7.68531343e-03
 -3.65057419e-03 -5.69706057e-03  8.10025421e-01  1.91012355e-02
  7.07049269e-02] [0.1955623  0.12778783 0.10316915 0.44456255 0.41230313 0.30430312
 0.40220988 0.41860727 0.3401073 ]
[ 5.73906856e-04 -6.18831527e-05  1.69143052e-04  7.68531343e-03
 -3.65057419e-03 -5.69706057e-03  8.10025421e-01  1.91012355e-02
  7.07049269e-02] [0.1955623  0.12778783 0.10316915 0.44456255 0.41230313 0.30430312
 0.40220988 0.41860727 0.3401073 ]
(8618, 128, 9) (8618, 7) (1356, 128, 9) (1356, 7)


In [23]:
model_list = [LSTM_CNN_SANDWICH,LSTM_CNN,ConvLSTM_PARALLEL,LSTM_CNN_PARALLEL]
model_name = ['LSTM_CNN_SANDWICH','LSTM_CNN','ConvLSTM_PARALLEL','LSTM_CNN_PARALLEL']

for model_no,model_func in enumerate(model_list):
    model = model_func(X_train.shape[1:],Y_train.shape[1])
    history = model.fit(X_train, Y_train, epochs=150, batch_size=32, validation_data=(X_val, Y_val),verbose=2,callbacks=callbacks(model_name[model_no], patience=30,
    location = "G:\\Project Server Pred\\4-2 project\\model-weights"))
    pickle.dump(history.history,open(os.path.join("G:\\Project Server Pred\\4-2 project\\model-weights", 'model_{}_norm_{}_h').format(model_name[model_no], "UCI"), 'wb'))
    

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 128, 9)]          0         
_________________________________________________________________
bidirectional_46 (Bidirectio (None, 128, 64)           10752     
_________________________________________________________________
activation_9 (Activation)    (None, 128, 64)           0         
_________________________________________________________________
reshape_36 (Reshape)         (None, 128, 64, 1)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 62, 64, 64)        384       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 31, 64, 64)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 14, 64, 32)        102

KeyboardInterrupt: 

In [12]:
model = LSTM_CNN(X_train.shape[1:],Y_train.shape[1])
model.fit(X_train, Y_train, epochs=150, batch_size=32, validation_data=(X_val, Y_val),verbose=2)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 128, 9)]          0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 128, 64)           10752     
_________________________________________________________________
activation_13 (Activation)   (None, 128, 64)           0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 128, 64)           24832     
_________________________________________________________________
activation_14 (Activation)   (None, 128, 64)           0         
_________________________________________________________________
reshape_9 (Reshape)          (None, 128, 64, 1)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 62, 30, 64)        1664

KeyboardInterrupt: 